# Hypothesis: Perfect Knowledge of Size and Complexity Improves Mean Latency
We assess the impact of perfect knowledge of query cost metrics on mean latency by comparing the following scheduling strategies:  
FIFO Baseline: Jobs are scheduled in a first-in, first-out order.   
SJR Baseline: Jobs are ordered in ascending order of \texttt{size} (i.e., the sum of the prompt, answer, and average reasoning trace length), approximating a shortest-job-remaining strategy.   
Proposed Approach: Jobs are sorted in ascending order of \texttt{size}', a combined cost metric that incorporates both the token count and the reasoning complexity.   


Load the GSM8K dataset from huggingface. For each problem in this dataset, to determine the per-query ground truth size and complexity, we first draw $K=100$ reasoning trajectories $R_i$ for $i \in \{1,...,100\}$ using vllm for serving. We test on the qwen 1.5b distilled deepseek model. For each reasoning trajectory, we extract the answer from "\boxed{}" and determine whether the answer is correct. Then, compute $p$ the proportion of reasoning trajectories which result in the correct answer. Let $C \in [0,1]$ be the desired accuracy threshold.  
Define the ground truth size to be the average |R_i| length of the number of tokens in the sampled reasoning trajectories. As in the self-constency paper, define the complexity to be the $n$ such that P(majority correct) = $\sum_{k=ceil(n/2)}^n {n \choose k} \cdot p^k (1-p)^{n-k} > C$.  
The final result of the code should be to construct a pandas dataframe that contains one row for each problem in GSM8K, and columns for the question text, answer, an array of the 100 sampled reasoning traces, the proportion p of the reasoning traces that yielded the correct answer, the computed ground truth "size", and the computed ground truth "complexity."  

Logistics:  
When sampling with vLLM, use temperature 0.6 and max_tokens 4096.  
Use the Together library to help with batched processing and parallelization if necessary.  
Use the together API key 'b6d3bd0350b68e70f37b355786f3808940fc279b16cc6c2ebd1b38781894f911'.   

In [1]:
import os
import re
import math
import numpy as np
import pandas as pd
from datasets import load_dataset

# -------------------------------
# Set up OpenAI client to use vLLM's API server.
# Assumes OPENAI_API_KEY is set in the environment.
from openai import OpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    #base_url=openai_api_base,
)

# Retrieve the model id from the available models.
models = client.models.list()
model = models.data[0].id  # or choose a specific model if needed
model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
#model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

In [2]:
# -------------------------------
# Function to check whether the extracted answer is correct.
def is_correct_answer(extracted_answer, ground_truth):
    """
    Determines if the extracted answer matches the ground truth answer.
    This example uses a simple string equality check.
    """
    if extracted_answer is None:
        return False
    return extracted_answer.strip() == ground_truth.strip()

# -------------------------------
# Function to compute the complexity.
def compute_complexity(p, C):
    """
    Computes the minimum integer n such that the probability that the majority of
    n independent samples are correct exceeds the threshold C.
    """
    n = 1
    while n<100: # set max required of 100 queries
        majority = math.ceil(n / 2)
        prob_majority = sum(
            math.comb(n, k) * (p ** k) * ((1 - p) ** (n - k))
            for k in range(majority, n + 1)
        )
        if prob_majority > C:
            return n
        n += 1
    return n

# -------------------------------
# Modified extract_answer function to return the last occurrence.
def extract_answer(trace):
    """
    Extract the last occurrence of the answer inside '\\boxed{...}' from the trace.
    """
    if "boxed" not in trace:
        return None
    answer = "}".join(trace.split("boxed{")[-1].split("}")[:-1])
    if "</think>" in answer:
        answer = answer.split("</think>")[0]
        answer = answer.split("}")[0] if "}" in answer else answer
    return answer
    #pattern = r'\\boxed\{([^}]+)\}'
    #matches = re.findall(pattern, trace)
    #if matches:
    #    return matches[-1].strip()
    #return None

In [3]:
# -------------------------------
# Load the GSM8K dataset from Hugging Face (using the "test" split as an example).
from datasets import load_dataset
# dataset = load_dataset("openai/gsm8k", "main")
dataset = load_dataset("HuggingFaceH4/MATH-500")
problem_key = "problem" if "problem" in dataset["test"][0].keys() else "question"

In [4]:
dataset["test"]

Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 500
})

In [5]:
dataset=dataset["test"][:100]
print(len(dataset[problem_key]))

100


In [6]:
import time
import numpy as np
from vllm import LLM, SamplingParams

# Desired accuracy threshold for computing complexity.
accuracy_threshold = 0.8  # modify as needed

# Suffix to be appended to each prompt.
chain_of_thought_suffix = " Please reason step by step, and put your final answer within \\boxed{}."
num_completions = 100      # Number of reasoning chains per prompt
batch_size = 4            # Adjust batch size as needed

# Set up the sampling parameters.
# Note: max_tokens may be set high if reasoning traces are long.
sampling_params = SamplingParams(temperature=0.6, top_p=0.95, max_tokens=4096, n=num_completions)

In [7]:
print(model)
# Initialize the vLLM engine with the desired model.
# TODO: figure out what model/dataset combo to use so that for most queries the probability is between 0.5 and 1.0
llm = LLM(model=model)

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
INFO 03-06 16:31:10 __init__.py:207] Automatically detected platform cuda.
INFO 03-06 16:31:27 config.py:549] This model supports multiple tasks: {'reward', 'score', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 03-06 16:31:27 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-06 16:31:27 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 03-06 16:31:28 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust

[W306 16:31:33.642767628 socket.cpp:752] [c10d] The client socket cannot be initialized to connect to [holygpu8a18204.rc.fas.harvard.edu]:42037 (errno: 97 - Address family not supported by protocol).


INFO 03-06 16:31:33 model_runner.py:1110] Starting to load model deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B...
INFO 03-06 16:31:33 weight_utils.py:254] Using model weights format ['*.safetensors']
INFO 03-06 16:31:33 weight_utils.py:304] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-06 16:31:34 model_runner.py:1115] Loading model weights took 3.3460 GB
INFO 03-06 16:31:35 worker.py:267] Memory profiling takes 0.91 seconds
INFO 03-06 16:31:35 worker.py:267] the current vLLM instance can use total_gpu_memory (79.22GiB) x gpu_memory_utilization (0.90) = 71.29GiB
INFO 03-06 16:31:35 worker.py:267] model weights take 3.35GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 1.41GiB; the rest of the memory reserved for KV Cache is 66.38GiB.
INFO 03-06 16:31:36 executor_base.py:111] # cuda blocks: 155355, # CPU blocks: 9362
INFO 03-06 16:31:36 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 18.96x
INFO 03-06 16:31:39 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.21it/s]

INFO 03-06 16:31:55 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.29 GiB
INFO 03-06 16:31:55 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 20.53 seconds


In [8]:
results = []

# Build a list of prompts from the dataset.
prompts = []
# Also store the ground truth answers (after any necessary processing).
ground_truths = []
for question, answer in zip(dataset[problem_key], dataset["answer"]):
        # Build prompt: append the chain-of-thought suffix.
        prompt = f"Question: {question}\nAnswer:" + chain_of_thought_suffix
        prompts.append(prompt)
        # Assuming ground truth answer is provided after "#### "
        ground_truth = answer.split("#### ")[1] if "#### " in answer else answer
        ground_truths.append(ground_truth)

latencies = []  # To record latency per batch

# Process prompts in batches.
#for i in range(0, len(prompts), batch_size):
for i in range(0, 20, batch_size):
        batch_prompts = prompts[i:i + batch_size]
        batch_ground_truths = ground_truths[i:i + batch_size]
        start_time = time.time()
        # Generate responses in batch.
        # Here we assume that passing n=num_completions in SamplingParams instructs vLLM to return
        # num_completions outputs per prompt. The returned object 'outputs' is assumed to be a list
        # of response objects corresponding to each prompt in the batch. Each response object should
        # have an attribute "outputs" that is a list of completions.
        batch_outputs = llm.generate(batch_prompts, sampling_params)
        end_time = time.time()
        batch_latency = end_time - start_time
        avg_latency = batch_latency / len(batch_prompts)
        latencies.append(avg_latency)
        print(f"Processed batch {i//batch_size+1}: {len(batch_prompts)} queries in {batch_latency:.4f} s (avg {avg_latency:.4f} s/query)")
        
        # Process each prompt's responses.
        for prompt_idx, response in enumerate(batch_outputs):
            traces = [choice.text for choice in response.outputs]  # Assuming each 'choice' has .text
            correct_count = 0
            token_lengths = []
            extracted_answers = []
            for trace in traces:
                # Count tokens using simple whitespace split.
                token_lengths.append(len(trace.split()))
                extracted = extract_answer(trace)
                extracted_answers.append(extracted)
                #if is_correct_answer(extracted, batch_ground_truths[prompt_idx]):
                #    correct_count += 1
            #p = correct_count / len(traces)  # Proportion of correct completions.
            avg_tokens = np.mean(token_lengths)
            #complexity = compute_complexity(p, accuracy_threshold)
            
            results.append({
                "prompt": batch_prompts[prompt_idx],
                "ground_truth_answer": batch_ground_truths[prompt_idx],
                "extracted_answers": extracted_answers,
                "reasoning_traces": traces,
                #"p": p,
                "avg_token_count": avg_tokens,
                #"complexity": complexity,
            })
    
#overall_mean_latency = sum(latencies) / len(latencies)
#print(f"\nMean latency per query: {overall_mean_latency:.4f} seconds")

Processed prompts:   1%|          | 4/400 [01:35<2:37:29, 23.86s/it, est. speed input: 3.14 toks/s, output: 9712.97 toks/s]


Processed batch 1: 4 queries in 95.5016 s (avg 23.8754 s/query)


Processed prompts:   1%|          | 4/400 [01:51<3:03:51, 27.86s/it, est. speed input: 4.89 toks/s, output: 9370.27 toks/s] 


Processed batch 2: 4 queries in 111.4620 s (avg 27.8655 s/query)


Processed prompts:   1%|          | 4/400 [02:24<3:58:54, 36.20s/it, est. speed input: 2.40 toks/s, output: 9205.58 toks/s]  


Processed batch 3: 4 queries in 144.8240 s (avg 36.2060 s/query)


Processed prompts:   1%|          | 4/400 [01:44<2:51:56, 26.05s/it, est. speed input: 5.92 toks/s, output: 9623.80 toks/s]


Processed batch 4: 4 queries in 104.2421 s (avg 26.0605 s/query)


Processed prompts:   1%|          | 4/400 [02:37<4:19:55, 39.38s/it, est. speed input: 3.26 toks/s, output: 9075.49 toks/s]  

Processed batch 5: 4 queries in 157.5650 s (avg 39.3912 s/query)


In [9]:
overall_mean_latency = sum(latencies) / len(latencies)
print(f"\nMean latency per query: {overall_mean_latency:.4f} seconds")


Mean latency per query: 30.6797 seconds


In [10]:
# Construct a pandas DataFrame from the results.
df = pd.DataFrame(results)
df["extracted_answers"] = df["reasoning_traces"].apply(lambda traces: [extract_answer(trace) for trace in traces])
#df.tail()

In [11]:
import os
import pandas as pd
import concurrent.futures
from openai import OpenAI

# -------------------------------------------------------------------
# Set up the local distilled deepseek model using OpenAI's API interface.
# Assumes OPENAI_API_KEY is set in your environment and that you have GPU access.
openai_api_key = os.getenv("OPENAI_API_KEY")
#openai_api_base = "http://localhost:8000/v1"

client = OpenAI(api_key=openai_api_key)#, base_url=openai_api_base)

# Get the model id (here we simply take the first model returned; adjust as necessary)
models = client.models.list()
model = models.data[0].id

# -------------------------------------------------------------------
# Define a function that uses the local model to determine if an extracted answer
# matches the ground truth answer.
def check_answer_match(extracted: str, ground_truth: str) -> bool:
    """
    Constructs a prompt comparing the extracted answer and the ground truth answer,
    and asks the model to answer "Yes" or "No" if they match.
    Returns True if the model indicates a match.
    """
    prompt = (
        f"Extracted answer: \"{extracted}\"\n"
        f"Ground truth answer: \"{ground_truth}\"\n"
        "Do they represent the same answer? Answer with Yes or No."
    )
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,  # Use deterministic output.
        max_tokens=10,
    )
    # Obtain the model's reply and convert to lowercase.
    reply = response.choices[0].message.content.strip().lower()
    # Return True if "yes" is in the reply.
    return "yes" in reply

# -------------------------------------------------------------------
# Define a function to compute the fraction of extracted answers that match the ground truth.
def compute_fraction(row) -> float:
    ground_truth = row["ground_truth_answer"]
    extracted_list = row["extracted_answers"]
    #print("compute_fraction")
    
    if not extracted_list:
        return 0.0  # Avoid division by zero.
    
    match_count = 0
    nonnone_answers = sum([1 for extracted in extracted_list if extracted is not None])
    # Use ThreadPoolExecutor to batch calls for each extracted answer.
    with concurrent.futures.ThreadPoolExecutor() as executor:
        if extracted is not None:
            nonnone_answers+=1
            futures = [executor.submit(check_answer_match, extracted, ground_truth) for extracted in extracted_list if extracted is not None]
            for future in concurrent.futures.as_completed(futures):
                if future.result():
                    match_count += 1
    
    p=(match_count / nonnone_answers) if (nonnone_answers>0) else 0
    print(p,match_count,nonnone_answers)
    return p

# -------------------------------------------------------------------
# Suppose you already have a DataFrame `df` with columns "ground_truth_answer" (string)
# and "extracted_answers" (list of strings). Compute the new column "p" using the model.
df["p"] = df.apply(compute_fraction, axis=1)

# Display the first few rows of the updated DataFrame.
print(df["p"])

0.9489795918367347 93 98
0.7450980392156863 38 51
0.9690721649484536 94 97
0.8651685393258427 77 89
0.7719298245614035 44 57
0.9550561797752809 85 89
0.8372093023255814 72 86
0.8043478260869565 74 92
0.84375 81 96
0.0 0 9
0.6444444444444445 29 45
0.0 0 50
0.7804878048780488 64 82
0.9587628865979382 93 97
0.7662337662337663 59 77
0.7261904761904762 61 84
0.9438202247191011 84 89
0.3333333333333333 4 12
0.125 1 8
0.391304347826087 27 69
0     0.948980
1     0.745098
2     0.969072
3     0.865169
4     0.771930
5     0.955056
6     0.837209
7     0.804348
8     0.843750
9     0.000000
10    0.644444
11    0.000000
12    0.780488
13    0.958763
14    0.766234
15    0.726190
16    0.943820
17    0.333333
18    0.125000
19    0.391304
Name: p, dtype: float64


In [12]:
df["complexity"] = df["p"].apply(lambda p: compute_complexity(p, 0.95))

In [13]:
df["extracted_answers"].iloc[0]

[None,
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '\\left(3, \\dfrac{\\pi}{2}\\right)',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '(\\sqrt{3}, \\frac{\\pi}{3})',
 '\\left(3, \\dfrac{\\pi}{2}\\right)',
 '\\left(3, \\dfrac{\\pi}{2}\\right)',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3,\\ \\frac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\dfrac{\\pi}{2}\\right)',
 '\\left(3, \\dfrac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '(3, \\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '(3,\\frac{\\pi}{2})',
 '(3, \\frac{\\pi}{2})',
 '\\left(3, \\frac{\\pi}{2}\\right)',
 '\\left(3, \\dfrac{\\pi}{2}\\righ

In [14]:
df["p"]

0     0.948980
1     0.745098
2     0.969072
3     0.865169
4     0.771930
5     0.955056
6     0.837209
7     0.804348
8     0.843750
9     0.000000
10    0.644444
11    0.000000
12    0.780488
13    0.958763
14    0.766234
15    0.726190
16    0.943820
17    0.333333
18    0.125000
19    0.391304
Name: p, dtype: float64

#### Use the ground truth size and complexity
Define `size'`  
Compute the mean latency if we sort in ascending order of size before batching.  

For the baselines: compute the min complexity such that the accuracy exceeds the desired threshold across all queries.

Baseline 1: random ordering of queries, fixed complexity for all.  
Baseline 2: sorted by `size` ordering of queries, fixed complexity for all.  
Intervention: sorted by `size'` ordering of queries, custom complexity for all.  

Define a function that takes in an ordering of queries, and either a fixed complexity for all queries or an array of complexities, and computes the mean latency for this batch.  

In determining which queries to test on, keep only queries in math500 where the probability of producing the correct answer exceeds the probability of producing an incorrect answer across all queries (> 0.5).  

Use threshold 0.95.  

In [15]:
print(df.shape)
df_filtered = df[df["p"]>0.5]
print(df_filtered.shape)

(20, 7)
(15, 7)


In [16]:
THRESH = 0.9

In [17]:
from scipy.stats import binom

def majority_accuracy(n, p):
    """
    Given n chains and individual correctness probability p,
    return the probability that the majority vote is correct.
    Majority threshold: floor(n/2) + 1.
    """
    m = (n // 2) + 1
    # P(X >= m) where X ~ Binom(n, p)
    return binom.sf(m - 1, n, p)

def find_required_complexity(df, target=THRESH, n_min=1, n_max=100):
    """
    Iterate over candidate n (number of chains) and return the smallest n such that
    the average majority accuracy across all rows is at least 'target'.
    """
    for n in range(n_min, n_max + 1):
        # Compute the majority accuracy for each row using its p.
        accuracies = df["p"].apply(lambda p: majority_accuracy(n, p))
        mean_accuracy = accuracies.mean()
        print(f"n = {n:2d}, Mean Majority Accuracy = {mean_accuracy:.4f}")
        if mean_accuracy >= target:
            return n, mean_accuracy
    return None, None

# Find the smallest constant complexity that achieves target accuracy.
complexity_constant, achieved_mean = find_required_complexity(df_filtered, target=THRESH)
if complexity_constant is not None:
    print(f"\nRequired complexity (number of chains) = {complexity_constant}, which yields an average accuracy of {achieved_mean:.4f}")
else:
    print("No candidate n in the range achieved the target accuracy.")

n =  1, Mean Majority Accuracy = 0.8374
n =  2, Mean Majority Accuracy = 0.7107
n =  3, Mean Majority Accuracy = 0.9103

Required complexity (number of chains) = 3, which yields an average accuracy of 0.9103


In [18]:
df_filtered.columns

Index(['prompt', 'ground_truth_answer', 'extracted_answers',
       'reasoning_traces', 'avg_token_count', 'p', 'complexity'],
      dtype='object')

In [19]:
df_filtered["prompt"].iloc[0]

'Question: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$\nAnswer: Please reason step by step, and put your final answer within \\boxed{}.'

In [33]:
import time
import numpy as np
import pandas as pd
from vllm import LLM, SamplingParams

def run_queries_with_required_n(df, required_n="complexity", batch_size=4):
    """
    Run batched queries with vLLM using a constant number of reasoning chains 
    or a variable number specified by a column in the DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing at least a "question" column.
        required_n (int or str): If int, use this constant number of reasoning chains 
                                 for each query. If str, it should be the name of the 
                                 column that provides the number of chains per query.
        batch_size (int): Number of queries to process in a single batch (only used if required_n is constant).
        
    Returns:
        outputs (list): List of outputs from vLLM (one per query).
        mean_latency (float): Mean latency per query (in seconds).
    """
    
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    
    # If required_n is a constant (int)
    if isinstance(required_n, int):
        constant_n = required_n
        prompts = df["prompt"].tolist()
        
        # Create sampling parameters with constant_n completions per query.
        sampling_params = SamplingParams(
            temperature=0.6, 
            top_p=0.95, 
            max_tokens=4096, 
            n=constant_n
        )
        
        # Initialize the vLLM engine with the desired model.
        #llm = LLM(model=model_name)
        
        latencies = []
        outputs = []
        
        # Process prompts in batches.
        for i in range(0, len(prompts), batch_size):
            batch = prompts[i:i + batch_size]
            start_time = time.time()
            batch_outputs = llm.generate(batch, sampling_params)
            end_time = time.time()
            batch_latency = (end_time - start_time) / len(batch)
            latencies.append(batch_latency)
            outputs.extend(batch_outputs)
            print(f"Processed batch {i // batch_size + 1}: {len(batch)} queries in {end_time - start_time:.4f} s (avg {batch_latency:.4f} s/query)")
        
        mean_latency = np.mean(latencies)
        print(f"\nMean latency per query (constant n={constant_n}): {mean_latency:.4f} seconds")
        return outputs, mean_latency
    
    # If required_n is a column name (str)
    elif isinstance(required_n, str):
        if required_n not in df.columns:
            raise ValueError(f"Column '{required_n}' not found in the DataFrame.")
        
        outputs = []
        latencies = []
        # Initialize the vLLM engine once.
        #llm = LLM(model=model_name)
        
        # Process each query individually, using the number of completions specified in the column.
        for idx, row in df.iterrows():
            prompt = row["prompt"]
            try:
                n_completions = int(row[required_n])
            except Exception as e:
                raise ValueError(f"Could not convert row[{required_n}] to int at index {idx}: {e}")
            
            sampling_params = SamplingParams(
                temperature=0.6, 
                top_p=0.95, 
                max_tokens=4096, 
                n=n_completions
            )
            
            start_time = time.time()
            # Generate for a single query (wrapped in a list).
            output = llm.generate([prompt], sampling_params)
            end_time = time.time()
            latency = end_time - start_time
            latencies.append(latency)
            outputs.append(output[0])
            print(f"Processed query {idx+1} with n={n_completions} in {latency:.4f} s")
        
        mean_latency = np.mean(latencies)
        print(f"\nMean latency per query (variable n from column '{required_n}'): {mean_latency:.4f} seconds")
        return outputs, mean_latency
    
    else:
        raise ValueError("Parameter 'required_n' must be either an integer or a string column name.")


In [22]:
outputs, mean_latency = run_queries_with_required_n(df_filtered, required_n=complexity_constant, batch_size=4)
print(outputs, mean_latency)

Processed prompts:  33%|███▎      | 4/12 [00:16<00:33,  4.20s/it, est. speed input: 17.87 toks/s, output: 1459.18 toks/s]


Processed batch 1: 4 queries in 16.7953 s (avg 4.1988 s/query)


Processed prompts:  33%|███▎      | 4/12 [00:17<00:34,  4.29s/it, est. speed input: 31.75 toks/s, output: 1532.83 toks/s]


Processed batch 2: 4 queries in 17.1706 s (avg 4.2926 s/query)


Processed prompts:  33%|███▎      | 4/12 [00:17<00:34,  4.28s/it, est. speed input: 22.50 toks/s, output: 1637.77 toks/s]


Processed batch 3: 4 queries in 17.1175 s (avg 4.2794 s/query)


Processed prompts:  33%|███▎      | 3/9 [00:17<00:34,  5.67s/it, est. speed input: 22.03 toks/s, output: 1507.02 toks/s]

Processed batch 4: 3 queries in 17.0270 s (avg 5.6757 s/query)

Mean latency per query (constant n=3): 4.6116 seconds
[RequestOutput(request_id=35, prompt='Question: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$\nAnswer: Please reason step by step, and put your final answer within \\boxed{}.', prompt_token_ids=[151646, 14582, 25, 7169, 279, 1459, 4930, 15, 11, 18, 15087, 304, 51424, 13934, 311, 24660, 13934, 13, 220, 11252, 697, 4226, 304, 279, 1352, 4930, 81, 26266, 15976, 98406, 1380, 400, 81, 861, 220, 15, 3, 323, 400, 15, 1124, 273, 1124, 15976, 366, 220, 17, 1124, 2493, 2418, 198, 16141, 25, 5209, 2874, 3019, 553, 3019, 11, 323, 2182, 697, 1590, 4226, 2878, 1124, 79075, 46391], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text="*\n\nAlright, so I have this problem: convert the point (0,3) from rectan

In [24]:
print(mean_latency)

4.611627126733462


In [29]:
df_filtered["size"]=df_filtered["avg_token_count"]

/tmp/ipykernel_7406/499397108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["size"]=df_filtered["avg_token_count"]


In [30]:
outputs_sjr, mean_latency_sjr = run_queries_with_required_n(df_filtered.sort_values("size"), required_n=complexity_constant, batch_size=4)
print(mean_latency_sjr)

Processed prompts:  33%|███▎      | 4/12 [00:15<00:31,  3.89s/it, est. speed input: 24.69 toks/s, output: 1401.55 toks/s]


Processed batch 1: 4 queries in 15.5550 s (avg 3.8887 s/query)


Processed prompts:  33%|███▎      | 4/12 [00:17<00:34,  4.32s/it, est. speed input: 15.69 toks/s, output: 1652.20 toks/s]


Processed batch 2: 4 queries in 17.2801 s (avg 4.3200 s/query)


Processed prompts:  33%|███▎      | 4/12 [00:18<00:37,  4.69s/it, est. speed input: 18.50 toks/s, output: 2184.24 toks/s]


Processed batch 3: 4 queries in 18.7627 s (avg 4.6907 s/query)


Processed prompts:  33%|███▎      | 3/9 [00:17<00:35,  5.98s/it, est. speed input: 33.64 toks/s, output: 1709.31 toks/s]

Processed batch 4: 3 queries in 17.9283 s (avg 5.9761 s/query)

Mean latency per query (constant n=3): 4.7189 seconds
4.718882178266843


In [32]:
df_filtered.columns

Index(['prompt', 'ground_truth_answer', 'extracted_answers',
       'reasoning_traces', 'avg_token_count', 'p', 'complexity', 'size'],
      dtype='object')

In [34]:
outputs_optimized, mean_latency_optimized = run_queries_with_required_n(df_filtered.sort_values("size"), required_n="complexity", batch_size=4)
print(mean_latency_optimized)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.23s/it, est. speed input: 60.45 toks/s, output: 301.92 toks/s]


Processed query 14 with n=1 in 3.2283 s


Processed prompts:  50%|█████     | 1/2 [00:06<00:06,  6.56s/it, est. speed input: 10.37 toks/s, output: 448.67 toks/s]


Processed query 1 with n=2 in 6.5572 s


Processed prompts:  50%|█████     | 1/2 [00:04<00:04,  4.95s/it, est. speed input: 10.91 toks/s, output: 510.18 toks/s]


...

Processed query 16 with n=8 in 20.3950 s


Processed prompts:  25%|██▌       | 1/4 [00:15<00:45, 15.06s/it, est. speed input: 23.85 toks/s, output: 605.04 toks/s]


Processed query 5 with n=4 in 15.0582 s


Processed prompts:   4%|▍         | 1/24 [00:21<08:19, 21.70s/it, est. speed input: 2.07 toks/s, output: 3851.24 toks/s]

Processed query 11 with n=24 in 21.7016 s

Mean latency per query (variable n from column 'complexity'): 11.3674 seconds
11.367351643244426
